# 例：RAG

Retrieval-Augmented Generation，增强检索生成。

## 0.准备
下面是一些工具函数。用来显示langchain的一些运行结果。

In [15]:
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import json

def show_documents(docs: list[Document]):
    from IPython.display import HTML, display
    html = ""
    html += "<ul style=\"list-style: none;\">"
    for doc in docs:
      html += "<li><div style=\"margin: 15px 0;  box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2); transition: 0.3s;\">"
      html+=f"<pre style=\"background-color: #eee; font-size: 10px; border: 1px dashed #ccc; padding: 5px;\">{json.dumps(doc.metadata, indent=2, ensure_ascii=False)}</pre>"
      html+=f"<pre style=\"background-color: #eff; padding: 5px;\">{doc.page_content}</pre>"
      html+="</div></li>"
    display(HTML(html))    

def show_messages(messages: list[BaseMessage]):
    from IPython.display import HTML, display
    html = ""
    html += "<ul style=\"list-style: none; margin: 5px 0;\">"
    for msg in messages:
      html += "<li><div style=\"margin: 15px 0;\">"
      match msg.type:
        case "ai":
            html += "<div style=\"text-align: right; font-size: 24px;\">🤖</div>"
            html+=f"<pre style=\"background-color: #eff; float: right; padding: 5px; width: fit-content; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2); transition: 0.3s;   border-radius: 5px;\">{msg.content}</pre>"
        case "human":
            html += "<div style=\"text-align: left;font-size: 24px;\">👨🏻</div>"
            html+=f"<pre style=\"background-color: #ffe; padding: 5px; width: fit-content; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2); transition: 0.3s; border-radius: 5px;\">{msg.content}</pre>"
        case _:
            html += f"<div style=\"text-align: left;font-size: 24px;\">{msg.type}</div>"
            html+=f"<pre style=\"background-color: #eee; padding: 5px;width: fit-content; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2); transition: 0.3s;\">{msg.content}</pre>"
      html+="</div></li>"
    display(HTML(html))

def show_answer(message: AIMessage):
    from IPython.display import HTML, display
    html = ""
    html += "<div style=\"background-color: #eee; padding: 5px;\">"
    html += f"<div style=\"font-size: 9px; color: #333;\">id={message.id}</div>"
    html += f"<pre style=\"background-color: transparent; border: 1px dash #ccc; padding: 5px; width: fit-content;\">{message.content}</pre>"
    html += f"<pre style=\"font-size: 9px;\">{json.dumps(message.response_metadata, indent=2)}</pre>"
    html += "</div>"
    display(HTML(html))


## 1. 创建知识库

假设有Markdown格式的文章，如下：

In [16]:
markdown_document = """
# 六元素について

生活、仕事、教育。日々の暮らしや社会に関わるさまざまなものが、年齢や性別、時間や場所から自由になる世界が実現できる。ITの可能性は、まだまだ広がります。

そんなITの力で私たちが目指す「感動」や「幸せ」は、すべての人がやりたいことに挑戦できる未来、成長できる未来、成功を目指せる未来。だからこそ、あなたが実現したい未来を実現するための
「プラットフォーム」になりたい。

私たち六元素は、「ITプロフェッショナル力」による顧客の課題解決と「社会課題」に挑む自社サービス開発で、顧客と社会に感動と幸せを創造する「チャレンジ・プラットフォームカンパニー」です。

## 経営理念 PHILOSOPHY

顧客へ感動を、社員へ幸せを。

## ミッション MISSION

ITの力で、感動と幸せを創造する。

## ビジョン VISION

「ITプロフェッショナル力」と「社会課題へ挑むベンチャーマインド」で、半歩先の未来価値を創造し続け、感動と幸せを提供する、「チャレンジ・プラットフォーム カンパニー」。

## バリュー VALUE

We are CHONPS（必須元素）

- Challenger やってみよう
- Heart 情熱と心を込めよう
- One team ワンチームになろう
- New technology 先端技術を取り入れよう
- Professional　プロでいよう
- Study 学び続けよう

## 社名由来 ORIGIN OF COMPANY NAME

### 六元素の意味

あらゆる生命は活動を維持するために酸素、炭素、窒素、水素、燐、硫黄という六つの元素が必要だとされています。私たちは、企業を発展させるためにも顧客、従業員、株主、社会、パートナー、ライバルという六つの元素が大切だと考えます。さらに中国では、まだ発見されていない未来の元素のことを「第六元素」とも言います。社名にはこれら二つの意味合いから、「堅実な基礎を築き、未来の価値を追求していく」という意味を込めています。

"""

### 1.1 切分

#### (1) 按章节切分大块

按照章节可以切分片段。

In [17]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "H1"),
    ("##", "H2"),
    ("###", "H3"),
]

# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=True
)
md_header_splits = markdown_splitter.split_text(markdown_document)

show_documents(md_header_splits)

#### (2) 清洗

清洗Markdown里面的标记，只保留文字部分。

In [18]:
from unstructured.partition.md import partition_md

for doc in md_header_splits:
  doc.page_content = text = "\n".join([str(e) for e in partition_md(text=doc.page_content)])

show_documents(md_header_splits)

#### (3) 按长度切分小块

进一步，可以按字符串长度切分

In [19]:
# Char-level splits
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 250
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(md_header_splits)

show_documents(splits)

### 1.2 建库

将切分后的文章片段，存入可以进行相关度检索的数据库中。

#### (1) BM25

In [20]:
from langchain_community.retrievers import BM25Retriever
from typing import List
from sudachipy import tokenizer
from sudachipy import dictionary

# 默认BM25是按照空格切分句子，这对日语是无效的。
# 这里使用sudachi (from ワークス徳島人工知能NLP研究所) 来分析日语。

def generate_word_ngrams(text, i, j, binary=False):
    """
    文字列を単語に分割し、指定した文字数のn-gramを生成する関数。

    :param text: 文字列データ
    :param i: n-gramの最小文字数
    :param j: n-gramの最大文字数
    :param binary: Trueの場合、重複を削除
    :return: n-gramのリスト
    """

    tokenizer_obj = dictionary.Dictionary(dict="full").create()
    mode = tokenizer.Tokenizer.SplitMode.A
    tokens = tokenizer_obj.tokenize(text ,mode)
    words = [token.surface() for token in tokens]

    ngrams = []
    
    for n in range(i, j + 1):
        for k in range(len(words) - n + 1):
            ngram = tuple(words[k:k + n])
            ngrams.append(ngram)
    
    if binary:
        ngrams = list(set(ngrams))  # 重複を削除
    
    return ngrams

def preprocess_func(text: str) -> List[str]:
    return generate_word_ngrams(text,1, 1, True)


retriever = BM25Retriever.from_documents(splits, preprocess_func=preprocess_func)

scores = retriever.vectorizer.get_scores(preprocess_func("六元素の意味は何ですか。"))
for text, score in zip(splits, scores):
    print(text, score)

chunks = retriever.invoke("六元素の意味は何ですか。")

show_documents(chunks)

page_content='生活、仕事、教育。日々の暮らしや社会に関わるさまざまなものが、年齢や性別、時間や場所から自由になる世界が実現できる。ITの可能性は、まだまだ広がります。 そんなITの力で私たちが目指す「感動」や「幸せ」は、すべての人がやりたいことに挑戦できる未来、成長できる未来、成功を目指せる未来。だからこそ、あなたが実現したい未来を実現するための 「プラットフォーム」になりたい。' metadata={'H1': '六元素について'} 0.6186190696709234
page_content='「プラットフォーム」になりたい。 私たち六元素は、「ITプロフェッショナル力」による顧客の課題解決と「社会課題」に挑む自社サービス開発で、顧客と社会に感動と幸せを創造する「チャレンジ・プラットフォームカンパニー」です。' metadata={'H1': '六元素について'} 3.3040027538216674
page_content='顧客へ感動を、社員へ幸せを。' metadata={'H1': '六元素について', 'H2': '経営理念 PHILOSOPHY'} 0.4279916943149802
page_content='ITの力で、感動と幸せを創造する。' metadata={'H1': '六元素について', 'H2': 'ミッション MISSION'} 0.7972234284189144
page_content='「ITプロフェッショナル力」と「社会課題へ挑むベンチャーマインド」で、半歩先の未来価値を創造し続け、感動と幸せを提供する、「チャレンジ・プラットフォーム カンパニー」。' metadata={'H1': '六元素について', 'H2': 'ビジョン VISION'} 0.586024670750067
page_content='We are CHONPS（必須元素） - Challenger やってみよう - Heart 情熱と心を込めよう - One team ワンチームになろう - New technology 先端技術を取り入れよう - Professionalプロでいよう - Study 学び続けよう' metadata={'H1': '六元素について', 'H2': 'バリュー VALUE'} 0.253901

Langchain的BM25检索器有个问题是，不论相似度高低，总是返回固定数量的片段。哪怕最高相似度为0，那么也返回相似度为0的片段。

下面自定义的检索器可以改善这个问题。通过设定一个最低的`score_threshold`，忽略掉相似度过低的片段。

In [21]:
from langchain_community.retrievers import BM25Retriever
from typing import List
from sudachipy import tokenizer
from sudachipy import dictionary

class BM25RetrieverWithScores(BM25Retriever):
    def get_relevant_documents_with_scores(self, query):
        processed_query = self.preprocess_func(query)
        scores = self.vectorizer.get_scores(processed_query)
        docs_and_scores = [(doc, score) for doc, score in zip(self.docs, scores)]
        sorted_docs_and_scores = sorted(docs_and_scores, key=lambda x: x[1], reverse=True)
        return sorted_docs_and_scores[:self.k]

    def _get_relevant_documents(self, query: str) -> List[Document]:
        processed_query = self.preprocess_func(query)
        scores = self.vectorizer.get_scores(processed_query)
        docs_and_scores = [(doc, score) for doc, score in zip(self.docs, scores)]
        docs_and_scores = sorted(docs_and_scores, key=lambda x: x[1], reverse=True)
        threshold = self.metadata.get("score_threshold", 0)
        docs = []
        for doc, score in docs_and_scores:
            doc.metadata["score"] = score
            if score > threshold:
                docs.append(doc)
        return docs[:self.k]

retriever = BM25RetrieverWithScores.from_documents(
    splits,
    preprocess_func=preprocess_func,
    metadata = {
        "score_threshold": 1.5
    }
)


## 2.提问

### 2.1 提示文

整体上，作为提示文模板的对话设计如下。

输入：
- `question`: 问题
- `documents`: 问题相关的参考资料。注意，因为langchain的限制，这里需要`list[BaseMessage]`类型。

输出:
- 提示文。它主要包含一个对话，格式为`list[BaseMessage]`。

In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ("ai", "What can I help you?"),
    ("human", "Please answer my question using given documents."),
    ("ai", "OK."),
    ("placeholder", "{documents}"),
    ("human", "Here is my question.\nQuestion: {question}")
])

prompt = prompt_template.invoke({
    "question": "What is your name?",
    "documents": [
        HumanMessage(content="Document 1 ..."),
        AIMessage(content="OK.")        
    ]
})

show_messages(prompt.messages)

检索得到的文章片段是langchain的`Document`类型。本例子中，作为输入的`documents`数据应该是消息列表`list[BaseMessage]`类型。

下面就实现了一个从`list[Document]`构造`list[BaseMessage]`的函数。

In [23]:
from langchain_core.runnables import RunnableLambda
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
# 将Documents一览变换成Message列表
def convert(docs: list[Document]) -> list[BaseMessage]:
    if len(docs) == 0:
      return [
          HumanMessage(content="Sorry. There is no documents found for my question. Please answer my questions without reference documents."),
          AIMessage(content="OK. I will answer your questions by original training data.")
      ]
    else:
      messages: list[BaseMessage] = []
      messages.extend([
          HumanMessage(content="The documents will send to you one by one."),
          AIMessage(content="OK. Please show them one by one.")
      ])
      for i, doc in enumerate(docs):
        meta = "\n".join(f"- {k}: {v}" for k, v in doc.metadata.items())
        messages.extend([
          HumanMessage(content=f"[Document {i+1}]\nMetadata: \n{meta} \n\nContent: \n {doc.page_content}"),
          AIMessage(content="OK.")
        ])
      messages.extend([
          HumanMessage(content=f"OK. All {len(docs)} documents are sent to you."),
          AIMessage(content=f"Thank you. I will answer your questions based on those {len(docs)} documents.")
      ])
      return messages

documents_parser = RunnableLambda(convert)

# 测试一下
messages = documents_parser.invoke(retriever.invoke("六元素の意味は何ですか。"))
show_messages(messages)

进一步改进，让其满足如下输入/输出的格式。

- 输入：
```
"What is your name?"
```

- 输出：
```json
{
    "question": "What is your name?",
    "documents": [
        // Document型的列表，表示相关文档片段一览
    ]
}

In [24]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

retriever_chain = RunnableParallel(
    question = RunnablePassthrough(),
    documents = RunnablePassthrough() | retriever | documents_parser
)

串联起来

In [25]:
prompt_builder = retriever_chain | prompt_template
prompt = prompt_builder.invoke("六元素の意味は何ですか。")

show_messages(prompt.messages)

### 2.2 LLM调用

最后一步是向LLM提问。

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

qa = prompt_builder | llm

ans = qa.invoke("六元素の意味は何ですか。")

show_answer(ans)

## 3.界面

### 3.1 Jupyter的简单界面

In [27]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

output = widgets.Output(layout={"height": "300px", "padding": "10px"})

user_input = widgets.Textarea(value='',
                              placeholder='Enter text here...',
                              description='User:',
                              rows=2,
                              layout=widgets.Layout(width='auto'))

# Create button for user to click to send their message
send_button = widgets.Button(description='送信')

conversation_history = []

def send_button_clicked(b):
    # Append user input to conversation history
    conversation_history.append(HumanMessage(content = user_input.value))    
    response = qa.invoke(user_input.value)
    conversation_history.append(response)

    with output:
        clear_output(wait = False)
        show_messages(conversation_history)
    
    # Clear user input box
    user_input.value = ''

send_button.on_click(send_button_clicked)

# Display everything
display(output, user_input, send_button)

Output(layout=Layout(height='300px', padding='10px'))

Textarea(value='', description='User:', layout=Layout(width='auto'), placeholder='Enter text here...', rows=2)

Button(description='送信', style=ButtonStyle())